In [1]:
################ IMPORT LIBRARIES ########################

from tensorflow.keras.layers import Activation,Flatten,Dense,Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import cv2
import os

####################################################

In [2]:
################ PARAMETERS ########################

path = 'D:\Thida Htwe\SampleProject\Fish_Data'
#80% TRAINING AND 20% TESTING
testRatio = 0.2 
valRatio = 0.2
global X_train,X_test,y_train,y_test
DNN_Model_score = []
CNN_Model_score = []

####################################################

In [3]:
#### IMPORTING DATA/IMAGES FROM DATA SET FOLDERS 

count = 0
images = []     # LIST CONTAINING ALL THE IMAGES 
classNo = []    # LIST CONTAINING ALL THE CORRESPONDING CLASS ID OF IMAGES 
myList = os.listdir(path)
print("Total Classes Detected:",len(myList))
noOfClasses = len(myList)
print("Importing Classes .......")
for x in range (0,noOfClasses):
    myPicList = os.listdir(path+"/"+str(x))
    for y in myPicList:
        curImg = cv2.imread(path+"/"+str(x)+"/"+y,cv2.IMREAD_GRAYSCALE)
        curImg = cv2.resize(curImg,(28,28))
        
        images.append(curImg)
        classNo.append(x)
    print(x,end= " ")
print(" ")
print("Total Images in Images List = ",len(images))
print("Total IDS in classNo List= ",len(classNo))

####################################################

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\Thida Htwe\\SampleProject\\Fish_Data'

In [ ]:
#### CONVERT IMAGES AND CLASS_NO TO NUMPY ARRAY 

images = np.array(images)
classNo = np.array(classNo)
print(images.shape)

####################################################

In [ ]:
#### SPLITTING THE DATA TO TRAIN DATA, TEST DATA,AND VALIDATION DATA
X_train,X_test,y_train,y_test = train_test_split(images,classNo,test_size=testRatio)
X_train,X_validation,y_train,y_validation = train_test_split(X_train,y_train,test_size=valRatio)
print(X_train.shape)
print(X_test.shape)
print(X_validation.shape)

In [ ]:
#### PLOT BAR CHART FOR DISTRIBUTION OF IMAGES
numOfSamples= []
for x in range(0,noOfClasses):
    print(len(np.where(y_train==x)[0]))
    numOfSamples.append(len(np.where(y_train==x)[0]))
print(numOfSamples)

plt.figure(figsize=(10,5))
plt.bar(range(0,noOfClasses),numOfSamples)
plt.title("No of Images for each Class")
plt.xlabel("Class ID")
plt.ylabel("Number of Images")
plt.show()

In [ ]:
# for visualisation purpose, Change the data into proper shape
plt.figure(figsize=(10,10))
for i in range(15):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(images[i])
    plt.xlabel(classNo[i])
plt.show()

In [ ]:
#### BUILD DNN MODEL

def build_dnn_model(hidden_layers,Neurons):
    m = Sequential()
    m.add(Flatten())
    for i in range(hidden_layers):
        m.add(Dense(Neurons[i],activation = 'relu'))
    m.add(Dense(10, activation = 'softmax'))
    m.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
    return m

####################################################

In [ ]:
#### BUILD CNN MODEL

def build_cnn_model():
    m = Sequential()
    m.add(Conv2D(128,(5,5),activation='relu'))
    m.add(MaxPooling2D(2,2))
    m.add(Conv2D(256,(5,5),activation='relu'))
    m.add(Conv2D(512,(5,5),activation='relu'))
    m.add(MaxPooling2D(2,2))
    m.add(Flatten())
    m.add(Activation('relu'))
    m.add(Dense(128,activation='relu'))
    m.add(Dense(10,activation = 'softmax'))
    m.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
    return m

####################################################

In [ ]:
#### TRAIN MODELS
def train(h_layers=None,neurons=None,b_size=None,eps=None):
    #### SPLITTING THE DATA TO TRAIN DATA, TEST DATA,AND VALIDATION DATA
    X_train,X_test,y_train,y_test = train_test_split(images,classNo,test_size=testRatio)
    X_train,X_validation,y_train,y_validation = train_test_split(X_train,y_train,test_size=valRatio)
    X_train = X_train/255.0
    X_test = X_test/255.0
    
    dnn_model = build_dnn_model(h_layers,neurons)
    print("\nLoading DNN dataset...")
    print("\nLoading DNN dataset Successful...")
    print("\nNormalizing DNN data...")
   
    print("\nNormalizing DNN data Successful...")
    os.system('cls')
    print("Training DNN model with : \nHidden Layers : ",h_layers,"\nNeurons : ,",neurons,"\nBatch Size : ",b_size,"\nEpochs : ",eps,"\n")
    dnn_result = dnn_model.fit(X_train,y_train,batch_size = b_size, epochs = eps, validation_data = (X_test,y_test))
    time.sleep(3)

    print("\nModel training Successful,DNN Model Details")
    print("\nDNN Model trained on ",len(X_train)," Samples and tested on ",len(X_test)," Samples");
    print("\nDNN Model Testing Accuracy : ", dnn_model.evaluate(X_test,y_test)[1])
    print("\nDNN Model Summary :")
    print(dnn_model.summary())
        
    tf.keras.utils.plot_model(dnn_model)

    pd.DataFrame(dnn_result.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
        
    #### PLOT THE RESULTS  
    plt.figure(1)
    plt.plot(dnn_result.history['loss'])
    plt.plot(dnn_result.history['val_loss'])
    plt.legend(['training','validation'])
    plt.title('Loss')
    plt.xlabel('epoch')
    plt.figure(2)
    plt.plot(dnn_result.history['accuracy'])
    plt.plot(dnn_result.history['val_accuracy'])
    plt.legend(['training','validation'])
    plt.title('Accuracy')
    plt.xlabel('epoch')
    plt.show()
        
    #### EVALUATE USING TEST IMAGES
    DNN_Model_score = dnn_model.evaluate(X_test,y_test,verbose=0)
    print('Test Score for DNN = ',DNN_Model_score[0])
    print('Test Accuracy for DNN =', DNN_Model_score[1]) 

    print("\nSaving DNN model...")
    dnn_model.save('DNN.h5')
      
    ### finish DNN
    
    ##start CNN
    X_train,X_test,y_train,y_test = train_test_split(images,classNo,test_size=testRatio)
    X_train,X_validation,y_train,y_validation = train_test_split(X_train,y_train,test_size=valRatio)
    X_train = X_train/255.0
    X_test = X_test/255.0
    
    cnn_model = build_cnn_model()
    print("\nLoading CNN dataset...")
    print("\nLoading CNN dataset Successful...")
    print("\nNormalizing CNN data...")
    print("\nNormalizing CNN data Successful...")
    print("\nConverting the input Image into Volume...")
    X_train = X_train.reshape(-1,28,28,1)
    X_test = X_test.reshape(-1,28,28,1)
    print("\nConvertion Successful...")
    os.system('cls')
    cnn_result = cnn_model.fit(X_train,y_train,batch_size = b_size, epochs = eps, validation_data = (X_test,y_test))
    time.sleep(3)
    os.system('cls')
    print("Model training Successful,CNN Model Details")
    print("\nCNN Model trained on ",len(X_train)," Samples and tested on ",len(X_test)," Samples");
    print("\nCNN Model Testing Accuracy : ", cnn_model.evaluate(X_test,y_test)[1])
    print("\nCNN Model Summary :")
    print(cnn_model.summary())
        
    tf.keras.utils.plot_model(cnn_model)
        
    pd.DataFrame(cnn_result.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
    
    #### PLOT THE RESULTS  
    plt.figure(1)
    plt.plot(cnn_result.history['loss'])
    plt.plot(cnn_result.history['val_loss'])
    plt.legend(['training','validation'])
    plt.title('Loss')
    plt.xlabel('epoch')
    plt.figure(2)
    plt.plot(cnn_result.history['accuracy'])
    plt.plot(cnn_result.history['val_accuracy'])
    plt.legend(['training','validation'])
    plt.title('Accuracy')
    plt.xlabel('epoch')
    plt.show()
        
    #### EVALUATE USING TEST IMAGES
    CNN_Model_score = cnn_model.evaluate(X_test,y_test,verbose=0)
    print('Test Score for CNN = ',CNN_Model_score[0])
    print('Test Accuracy for CNN =', CNN_Model_score[1]) 

    print("\nSaving CNN model...")
    cnn_model.save('CNN.h5')
    ##fishin cnn
    
    ### Compare two model ###
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_axes([1.2, 1.2, 1.2, 1.2])
    
    
    accuracies = [DNN_Model_score[1], CNN_Model_score[1]]
    models = ['DNN', 'CNN']
    ax.bar(models, accuracies, color=['lightseagreen', 'seagreen'])
   
    for i in ax.patches:
       height = i.get_height()
       ax.annotate('{}'.format(round(height, 4)),
                   xy=(i.get_x() + i.get_width() / 2, height),
                   xytext=(0, 3),  # 3 points vertical offset
                   textcoords="offset points",
                   ha='center', va='bottom')
    ax.yaxis.grid()
    plt.yticks(range(0, 1, 1))
    plt.title("Accuracy comparison of DNN and CNN in Image Classification Task")
    plt.savefig('accuracy_comparison_img_classification_ann_v_cnn.png')
    plt.show()
    plt.draw()
        
    
    ####################################################

In [ ]:
#### SHOW IMAGE RESULT

def draw(network,img,prediction):
    if (network == 1):
        print('This Image is Test with DNN')
    else: 
        print('This Image is Test with CNN')
        
    categorys = []
    ax1 = plt.subplot2grid((4,4),(0,0),rowspan = 1, colspan = 4)
    ax2 = plt.subplot2grid((4,4),(1,0),rowspan = 3, colspan = 4)
    ax1.imshow(img,cmap = plt.cm.binary)
    ax2.bar(range(10),prediction[0])
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax2.set_xticks(range(10))
    ax2.set_yticks([0.0,0.2,0.4,0.6,0.8,1.0])
    plt.show()
    
   
        
        
    if (np.argmax(prediction) == 0):
        print('This Image is Fish Dascyllus reticulatus')
    elif (np.argmax(prediction) == 1):
        print('This Image is Fish Plectroglyphidodondickil')
    elif (np.argmax(prediction) == 2):
        print('This Image is Fish Chromis chrysura')
    elif (np.argmax(prediction) == 3):
        print('This Image is Fish Amphiprion clarkii')
    elif (np.argmax(prediction) == 4):
        print('This Image is Fish Chaetodon trifascialis')
    else:
        print('This Image is Fish Nothing')

    
####################################################

In [ ]:
#### PREDICT IMAGES

def predict_images(network,model):
    testImg = "test.png"
    img_arr = cv2.imread(testImg,cv2.IMREAD_GRAYSCALE)
    
    img = cv2.imread("test.png",cv2.IMREAD_GRAYSCALE)
    img_arr = cv2.resize(img_arr,(28,28))
    img_arr = img_arr/255.0
    img_arr = np.asarray(img_arr)
    if(network == 1):
        reshaped_img = img_arr.reshape(-1,28,28)
    elif(network == 2):
        reshaped_img = img_arr.reshape(-1,28,28,1)
    predictions = model.predict(reshaped_img)

    draw(network,img_arr,predictions)
####################################################

In [ ]:
#### MAIN 

def main():
    os.system('cls')
    use_network = int(input("Select :\n\t1.Train and Compare DNN and CNN\n\t2.Test with Image: "))
    if(use_network != ""):
        if(use_network == 1):
            hidden_layers = int(input("\nEnter No. of hidden_layers in your Network : "))
            Neurons = []
            for i in range(hidden_layers):
                print("\nEnter No of Neurons in Hidden Layer ",(i+1)," : ",end="")
                Neurons.append(int(input()))
            e = int(input("\nEnter No. of epochs (Best 100 Unless you have a large dataset) : "))
            b = int(input("\nEnter the batch_size (Best 32 Unless you have a large dataset) : "))
            train(h_layers = hidden_layers, neurons = Neurons,b_size = b, eps = e)

                 
        elif(use_network == 2):
            dnn_model = keras.models.load_model('DNN.h5')
            predict_images(use_network,dnn_model)
                
            cnn_model = keras.models.load_model('CNN.h5')
            predict_images(use_network,cnn_model)

        else:
            print("Invalid Input, Rolling back in a moment")
            time.sleep(2)
            main()
    else:
        print("Invalid Input, Rolling back in a moment")
        time.sleep(2)
        main()
        
####################################################

In [ ]:
if __name__ == "__main__":
    main() 

In [ ]:
main()